## Integrating EvaDB with Plaid for Financial Insights
The goal of this project is to try and connect a bank account to a database. We will be using Plaid to access transactions, and then storing them in a Postgres DB and then use EvaDB to provide a natural language AI layer to our DB. 

### Connecting the Postgres Server to EvaDB

In [2]:
import evadb

cursor = evadb.connect().cursor()

If this is your first time running through this code, make sure to create the database below. Otherwise, you can skip this step

In [2]:
params = {
    "database": "evadb.db"
}
query = f"CREATE DATABASE plaid_data WITH ENGINE = 'sqlite', PARAMETERS = {params};"
cursor.query(query).df()

10-16-2023 18:07:05 ERROR [plan_executor:plan_executor.py:execute_plan:0186] plaid_data already exists.
Traceback (most recent call last):
  File "/Users/varunhegde/Documents/Georgia Tech/Fall 2023/CS 4420/plaid_evadb/evadb-venv/lib/python3.10/site-packages/evadb/executor/plan_executor.py", line 182, in execute_plan
    yield from output
  File "/Users/varunhegde/Documents/Georgia Tech/Fall 2023/CS 4420/plaid_evadb/evadb-venv/lib/python3.10/site-packages/evadb/executor/create_database_executor.py", line 42, in exec
    raise ExecutorError(f"{self.node.database_name} already exists.")
evadb.executor.executor_utils.ExecutorError: plaid_data already exists.


ExecutorError: plaid_data already exists.

Let's create our transactions table that we'll use to store our data. The information we're storing will be:
- The transaction id
- The dollar amount
- The category of the transaction
- The date
- The merchant name given to use from Plaid (possibly different than the original merchant name, since Plaid recategorizes it for simplicity)

In [144]:
cursor.query("""
    USE plaid_data {        
        DROP TABLE transactions
    }
""").df()

""


In [145]:
cursor.query("""
    USE plaid_data {        
        CREATE TABLE transactions (
            id TEXT PRIMARY KEY,
            amount FLOAT,
            category TEXT,
            date TEXT,
            merchant_name TEXT
        )
    }
""").df()

""


In [146]:
cursor.query("""
    USE plaid_data {
        SELECT * FROM transactions
    }
""").df()

,id,amount,category,date,merchant_name


### Integrating Plaid API

Make sure the API server is running

Let's test whether the API connection works! The following should print out "Hello World"

In [34]:
import requests
import json

resp = requests.get('http://127.0.0.1:8000/api')
print(resp.content)

b'Hello World'


Make sure to login and authenticate a bank account on the frontend (Check the README for instructions on how to run the frontend).

Sign in with a Bank Account and make sure both "Authentication Successful" messages show up

In [148]:
transactions = requests.get('http://127.0.0.1:8000/get_transactions')
data = json.loads(transactions.text)
for transac in data['transactions']:
    id = transac['transaction_id']
    amount = transac['amount']
    category = transac['category'][0]
    date = transac['date']
    merchant_name = transac['merchant_name']
    query = f"""
        USE plaid_data {{
            INSERT INTO transactions (id, amount, category, date, merchant_name) VALUES ("{id}", {amount}, "{category}", "{date}", "{merchant_name}")
        }}
    """
    cursor.query(query).df()

In [149]:
cursor.query("""
    USE plaid_data {
        SELECT * from transactions
    }
""").df()

,id,amount,category,date,merchant_name
0,WjqLGVylAahJa1rJVqoNiA5kRgxk1dSdEBz9g,5.40,Travel,"Tue, 30 May 2023 00:00:00 GMT",Uber
1,AAyx36kMWLfE3zqEkXN9hw6vM5ovyEhbPBNjd,-500.00,Travel,"Sun, 28 May 2023 00:00:00 GMT",United Airlines
2,G9KyPAx86WFaProa4bLqhamb9ZEbrquV3ZgBX,12.00,Food and Drink,"Sat, 27 May 2023 00:00:00 GMT",McDonald's
3,nlxMVeQg9yTWD53WaRlZhjyxVM5xpDuabJqew,4.33,Food and Drink,"Sat, 27 May 2023 00:00:00 GMT",Starbucks
4,bkP6VrzNoLhWe5xWNjKphZ8rEgarzViRqBpxZ,89.40,Food and Drink,"Fri, 26 May 2023 00:00:00 GMT",FUN
5,mR3gV7XowmIVMrQVJAR8tDbvpBrvWVcGJrZmJ,6.33,Travel,"Sat, 13 May 2023 00:00:00 GMT",Uber
6,yNJMlyWvR4UWy8xWe1NdhzEDN54DKWcJAgR3w,5.40,Travel,"Sun, 30 Apr 2023 00:00:00 GMT",Uber
7,91ydnDM4jPHRyzjRb9rLuybEJ7rEkdH7DKNG9,-500.00,Travel,"Fri, 28 Apr 2023 00:00:00 GMT",United Airlines
8,vdeMVyWmJxSApgzAMnd9hM1ZRqgZWeuLRwQzb,12.00,Food and Drink,"Thu, 27 Apr 2023 00:00:00 GMT",McDonald's
9,RdjQg6ykAzS8z7b8jw1xFjyo9vKo7gu64ZVlk,4.33,Food and Drink,"Thu, 27 Apr 2023 00:00:00 GMT",Starbucks


In [279]:
from dotenv import load_dotenv
load_dotenv()

True

In [280]:
import os
os.environ["OPENAI_KEY"] = os.getenv('OPEN_AI_KEY')

In [283]:
cursor.query("""
SELECT ChatGPT(
    "Write out a sentence for each row that uses the following template. Do not deviate from this whatsoever: [Merchant Name]: $[amount]", merchant_name, amount
)
FROM plaid_data.transactions;
""").df()

10-17-2023 14:43:03 ERROR [plan_executor:plan_executor.py:execute_plan:0186] 5.4 is not of type 'string' - 'messages.0.content'
Traceback (most recent call last):
  File "/Users/varunhegde/Documents/Georgia Tech/Fall 2023/CS 4420/plaid_evadb/evadb-venv/lib/python3.10/site-packages/evadb/executor/plan_executor.py", line 182, in execute_plan
    yield from output
  File "/Users/varunhegde/Documents/Georgia Tech/Fall 2023/CS 4420/plaid_evadb/evadb-venv/lib/python3.10/site-packages/evadb/executor/project_executor.py", line 45, in exec
    batch = apply_project(batch, self.target_list, self.catalog())
  File "/Users/varunhegde/Documents/Georgia Tech/Fall 2023/CS 4420/plaid_evadb/evadb-venv/lib/python3.10/site-packages/evadb/executor/executor_utils.py", line 46, in apply_project
    batches = [expr.evaluate(batch) for expr in project_list]
  File "/Users/varunhegde/Documents/Georgia Tech/Fall 2023/CS 4420/plaid_evadb/evadb-venv/lib/python3.10/site-packages/evadb/executor/executor_utils.py", 

ExecutorError: 5.4 is not of type 'string' - 'messages.0.content'

In [160]:
cursor.query("""
CREATE FUNCTION IF NOT EXISTS TextSummarizer
TYPE HuggingFace
TASK 'summarization'
MODEL 'facebook/bart-large-cnn';
""").df()

Your max_length is set to 142, but your input_length is only 11. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=5)


,0
0,Function TextSummarizer added to the database.


In [258]:
cursor.query("""
    DROP FUNCTION IF EXISTS ConcatenateColumnsTogether;
""").df()

,0
0,Function ConcatenateColumnsTogether successful...


In [259]:
cursor.query("""
    CREATE FUNCTION ConcatenateColumnsTogether IMPL 'functions/concat_columns.py';
""").df()

,0
0,Function ConcatenateColumnsTogether added to t...


In [260]:
cursor.query("""
    SELECT * FROM plaid_data.transactions;
""").df()

,transactions.id,transactions.amount,transactions.category,transactions.date,transactions.merchant_name
0,WjqLGVylAahJa1rJVqoNiA5kRgxk1dSdEBz9g,5.40,Travel,"Tue, 30 May 2023 00:00:00 GMT",Uber
1,AAyx36kMWLfE3zqEkXN9hw6vM5ovyEhbPBNjd,-500.00,Travel,"Sun, 28 May 2023 00:00:00 GMT",United Airlines
2,G9KyPAx86WFaProa4bLqhamb9ZEbrquV3ZgBX,12.00,Food and Drink,"Sat, 27 May 2023 00:00:00 GMT",McDonald's
3,nlxMVeQg9yTWD53WaRlZhjyxVM5xpDuabJqew,4.33,Food and Drink,"Sat, 27 May 2023 00:00:00 GMT",Starbucks
4,bkP6VrzNoLhWe5xWNjKphZ8rEgarzViRqBpxZ,89.40,Food and Drink,"Fri, 26 May 2023 00:00:00 GMT",FUN
5,mR3gV7XowmIVMrQVJAR8tDbvpBrvWVcGJrZmJ,6.33,Travel,"Sat, 13 May 2023 00:00:00 GMT",Uber
6,yNJMlyWvR4UWy8xWe1NdhzEDN54DKWcJAgR3w,5.40,Travel,"Sun, 30 Apr 2023 00:00:00 GMT",Uber
7,91ydnDM4jPHRyzjRb9rLuybEJ7rEkdH7DKNG9,-500.00,Travel,"Fri, 28 Apr 2023 00:00:00 GMT",United Airlines
8,vdeMVyWmJxSApgzAMnd9hM1ZRqgZWeuLRwQzb,12.00,Food and Drink,"Thu, 27 Apr 2023 00:00:00 GMT",McDonald's
9,RdjQg6ykAzS8z7b8jw1xFjyo9vKo7gu64ZVlk,4.33,Food and Drink,"Thu, 27 Apr 2023 00:00:00 GMT",Starbucks


In [265]:
cursor.query("""
    CREATE TABLE summary_table AS 
        SELECT ConcatenateColumnsTogether(*) FROM plaid_data.transactions;
""").df()

,0
0,The table summary_table has been successfully ...


In [320]:
cursor.query("""
    SELECT summaries FROM summary_table;
""").df()

,summary_table.summaries
0,"Spent $5.4 on the date Tue, 30 May 2023 00:00:..."
1,"Spent $-500.0 on the date Sun, 28 May 2023 00:..."
2,"Spent $12.0 on the date Sat, 27 May 2023 00:00..."
3,"Spent $4.33 on the date Sat, 27 May 2023 00:00..."
4,"Spent $89.4 on the date Fri, 26 May 2023 00:00..."
5,"Spent $6.33 on the date Sat, 13 May 2023 00:00..."
6,"Spent $5.4 on the date Sun, 30 Apr 2023 00:00:..."
7,"Spent $-500.0 on the date Fri, 28 Apr 2023 00:..."
8,"Spent $12.0 on the date Thu, 27 Apr 2023 00:00..."
9,"Spent $4.33 on the date Thu, 27 Apr 2023 00:00..."


In [317]:
cursor.query("""
    SELECT s.summaries FROM summary_table AS s JOIN plaid_data.transactions AS t WHERE s.ids = t.id AND t.category = 'Travel'
""").df()

,s.summaries
0,"Spent $5.4 on the date Tue, 30 May 2023 00:00:..."
1,"Spent $-500.0 on the date Sun, 28 May 2023 00:..."
2,"Spent $6.33 on the date Sat, 13 May 2023 00:00..."
3,"Spent $5.4 on the date Sun, 30 Apr 2023 00:00:..."
4,"Spent $-500.0 on the date Fri, 28 Apr 2023 00:..."
5,"Spent $6.33 on the date Thu, 13 Apr 2023 00:00..."
6,"Spent $5.4 on the date Fri, 31 Mar 2023 00:00:..."
7,"Spent $-500.0 on the date Wed, 29 Mar 2023 00:..."
8,"Spent $6.33 on the date Tue, 14 Mar 2023 00:00..."
9,"Spent $5.4 on the date Wed, 01 Mar 2023 00:00:..."


In [306]:
cursor.query("""
    CREATE OR REPLACE FUNCTION Summarize IMPL 'functions/summarize.py';
""").df()

,0
0,Function Summarize overwritten.


In [309]:
cursor.query("""
    CREATE TABLE new_test AS SELECT Summarize(summaries) FROM summary_table
""").df()

,0
0,The table new_test has been successfully creat...


Let's define a function that allows us to easily input a query and receive a response using ChatGPT on our EvaDB database.

In [376]:
def ask_question(query):
    final_query = f"""
    SELECT ChatGPT(
        "{query}. Make sure you do not return code or calculations. Return a straight answer that directly answers the question",
        total_summary
    ) FROM new_test
    """
    ans = cursor.query(final_query).df()
    return ans['chatgpt.response'].iloc[0]
    

In [356]:
q = 'Pretend you are a highly skilled financial advisor. Based on the spending data, taking into account categories, merchants, and the amount spent on each item, provide a few sentences on how you could budget better and what points you are spending a lot of money on.'
ask_question(q)

"Based on the spending data provided, it appears that a significant portion of your expenses is allocated towards travel, specifically on Uber and United Airlines. To budget better, you could consider exploring alternative transportation options or comparing prices between different airlines to potentially reduce your travel expenses. Additionally, it seems that you are spending a considerable amount on food and drink, particularly at McDonald's, Starbucks, and FUN. To save money in this category, you could consider cooking at home more often or exploring more affordable dining options. By making these adjustments, you can allocate your funds more efficiently and potentially save money in the long run."

In [355]:
q = """
In a few sentences, describe my general spending habits for different categories. Talk about total amount of money spent on each
merchant, and categories, and bring up interesting facts and insights. An example of this is: "You frequently spent money on travel, 
like the $435 you spent on Ubers last month, and you can maybe cut back on fast food spending, with $200 spent at McDonalds". This is
just an example, but stick to this format with the answer.
"""
ask_question(q)

"Based on the provided context, it appears that you have been spending a significant amount on travel, particularly on Uber and United Airlines. In the past month, you spent a total of $16.06 on Uber rides, with the highest expenditure being $6.33 on a single ride. Additionally, you spent a total of $1500 on United Airlines, with each transaction being -$500. \n\nIn terms of food and drink, you have been frequenting McDonald's, Starbucks, and FUN. Over the past month, you spent a total of $24.66 on McDonald's, with each transaction being $12.0. Similarly, you spent $8.66 on Starbucks, with each transaction being $4.33. Lastly, you spent $178.8 on FUN, with the highest expenditure being $89.4.\n\nBased on these spending habits, it seems like you have been prioritizing travel expenses, especially with the high expenditure on Uber and United Airlines. However, you may want to consider cutting back on fast food spending, as you have spent a significant amount at McDonald's."

In [373]:
q = "How much did I spend in total in the month of January? Don't show me your calculation process, just a straight answer"
ask_question(q)

'You spent a total of $-500.0 in the month of January.'

In [379]:
q = "How much money did I spend on Ubers in June?"
ask_question(q)

'You spent $5.4 on Ubers in June.'